# Extreme Wind Compound Risk — Interactive Demo

> **Beyond peak gusts: recovery windows and system margins matter more.**

This notebook walks through the full pipeline step-by-step:
1. Load a synthetic dataset
2. Compute hourly metrics (EWS, CWL, SHWe)
3. Aggregate to daily metrics and compound strain flags
4. Classify risk states
5. Visualise all five panels

In [ ]:
import sys
from pathlib import Path

# Make sure src/ is importable
SRC = Path('..').resolve() / 'src'
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 110

## 1. Load the cyclone dataset

In [ ]:
import data_loader

DATA_DIR = Path('..') / 'data'
df = data_loader.load(DATA_DIR / 'sample_cyclone_event.csv')
print(f"Loaded {len(df):,} hourly rows  ({df.index.min().date()} → {df.index.max().date()})")
df.head()

## 2. Compute hourly metrics

In [ ]:
import metrics

hourly, daily = metrics.compute_all(df)
print('Hourly columns:', list(hourly.columns))
hourly[['wind_speed_ms', 'gust_ms', 'EWS', 'CWL_hour', 'SHWe_hour']].head(12)

## 3. Daily aggregation — compound strain

In [ ]:
daily[['daily_CWL', 'daily_SHWe', 'high_wind_day',
       'failed_recovery_night', 'compound',
       'consecutive_compound_cycles']]

## 4. Risk state classification

In [ ]:
import risk_states

daily = risk_states.compute_risk_states(daily)
daily[['daily_CWL', 'daily_SHWe', 'consecutive_compound_cycles',
       'risk_state', 'risk_multiplier']]

## 5. Five-panel visualisation

In [ ]:
import visualization

fig = visualization.plot_all(
    hourly, daily,
    title_prefix='Cyclone Event',
    output_dir=Path('..') / 'output'
)
plt.show()

## 6. Compare all three scenarios

In [ ]:
scenarios = {
    'Cyclone Event':        DATA_DIR / 'sample_cyclone_event.csv',
    'Fire-Weather Period':  DATA_DIR / 'sample_wind_fireweather.csv',
    'Future +10% Scenario': DATA_DIR / 'sample_future_plus10pct_winds.csv',
}

for label, path in scenarios.items():
    d = data_loader.load(path)
    h, dy = metrics.compute_all(d)
    dy = risk_states.compute_risk_states(dy)
    max_rm = dy['risk_multiplier'].max()
    peak_state = dy.loc[dy['risk_multiplier'].idxmax(), 'risk_state']
    print(f"{label:<28}  peak multiplier={max_rm:5.2f}×   peak state={peak_state}")

## 7. Using your own data

```python
df = data_loader.load('/path/to/my_event.csv')
hourly, daily = metrics.compute_all(df)
daily = risk_states.compute_risk_states(daily)
fig = visualization.plot_all(hourly, daily, title_prefix='My Event')
plt.show()
```

Your CSV must have `timestamp` (ISO 8601), `wind_speed_ms`, and `gust_ms` columns.  
Optional: `rainfall_mm`, `fuel_dryness_index`, `infrastructure_vulnerability`.